<a href="https://colab.research.google.com/github/yerimming/gachon/blob/main/CV_AL_test8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   optimizer로 RAdam 사용
2.   lr = 0.0001



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
!pip install git+https://github.com/LiyuanLucasLiu/RAdam.git

  Cloning https://github.com/LiyuanLucasLiu/RAdam.git to /tmp/pip-req-build-9wbk09dr
  Running command git clone --filter=blob:none --quiet https://github.com/LiyuanLucasLiu/RAdam.git /tmp/pip-req-build-9wbk09dr
  Resolved https://github.com/LiyuanLucasLiu/RAdam.git to commit d9fd30a337894c4003768561d45e8730dbd41333
  Preparing metadata (setup.py) ... done
  Created wheel for RAdam: filename=RAdam-0.0.1-py3-none-any.whl size=7098 sha256=89161065c25bc162f0934a916fd09ce3f04b1a9e140ba4798437624e26ba2b77
  Stored in directory: /tmp/pip-ephem-wheel-cache-38e5rncq/wheels/27/dd/f4/a154a2dd1d807820314f09ba2fa30f13f96a0d2830e8bab05e
Successfully built RAdam


In [12]:
import os
import time

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

# radam 사용
from radam import RAdam

from PIL import Image
#내가 추가
import re

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [13]:
### Custom Dataset ###
class CUB2011(Dataset):
  def __init__(self, transform, mode='train'):
    self.transform = transform
    self.mode = mode

    if self.mode == 'train':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/train')
    elif self.mode == 'valid':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/valid')
    elif self.mode == 'test':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets/test')

  def __len__(self):
    return len(self.image_folder)

  def __getitem__(self, idx):
    img_path = self.image_folder[idx]
    img = Image.open(os.path.join('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/CUB_200_2011_repackage_class50/datasets', self.mode, img_path)).convert('RGB')
    img = self.transform(img)
    label = img_path.split('_')[-1].split('.')[0]
    label = re.sub(r'\([^)]*\)', '', label)
    label = int(label)
    return (img, label)

In [17]:
### Data Preprocessing ###

'''
# 데이터 정규화 추가
# 정규화 변환 정의
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
'''
transforms_train = transforms.Compose([transforms.Resize((448,448)),
                                       transforms.ToTensor()])
transforms_valtest = transforms.Compose([transforms.Resize((448,448)),
                                         transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train',
                    transform=transforms_train)
# transforms_train을 transforms_valtest로 변경
val_set = CUB2011(mode='valid',
                  transform=transforms_valtest)
test_set = CUB2011(mode='test',
                  transform=transforms_valtest)

print('Num of each dataset: ',len(train_set),len(val_set),len(test_set))

train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)

print("Loaded dataloader")

### Model / Optimizer ###
EPOCH = 30
lr = 0.001

model = models.resnet18(pretrained=True)

### Tranfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,50)
model.to(DEVICE)

optimizer = optim.RAdam(model.parameters(),lr=lr)

print("Created a learning model and optimizer")

Num of each dataset:  2362 296 298
Loaded dataloader
Created a learning model and optimizer


In [15]:
### Train/Evaluation ###
def train(model,train_loader,optimizer,epoch):
  model.train()
  for i,(image,target) in enumerate(train_loader):
    image,target = image.to(DEVICE),target.to(DEVICE)
    output = model(image)
    optimizer.zero_grad()
    # loss func을 어떤 것을 사용할 것인지?
    train_loss = F.cross_entropy(output,target).to(DEVICE)

    train_loss.backward()
    optimizer.step()

    if i%10 ==0:
      print(
          f'Train Epoch: {epoch} [{i}/{len(train_loader)}]\tloss: {train_loss.item():6f}')

  return train_loss

def evaluate(model,val_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for i,(image,target) in enumerate(val_loader):
      image,target = image.to(DEVICE),target.to(DEVICE)
      output = model(image)

      eval_loss += F.cross_entropy(output,target, reduction='sum').item()
      pred = output.max(1,keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

  eval_loss /= len(val_loader.dataset)
  eval_accuracy = 100*correct / len(val_loader.dataset)
  return eval_loss,eval_accuracy

In [18]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
  train_loss = train(model,train_loader,optimizer,epoch)
  val_loss,val_accuracy = evaluate(model,val_loader)

  # Save best model
  if val_accuracy > best:
    best = val_accuracy
    torch.save(model.state_dict(),"./best_model.pth")

  print(f"[{epoch}]Validation Loss: {val_loss:.4f},Accuracy: {val_accuracy:.4f}%")

# Test result
test_loss,test_accuracy = evaluate(model,test_loader)
print(f'[FINAL] Test Loss: {test_loss:.4f},Accuracy: {test_accuracy:.4f}%')

end = time.time()
elasped_time = end - start

print("Best Accuracy: ",best)
print(
    f"Elasped Time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")
print(
    f"time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")

Train Epoch: 0 [0/74]	loss: 4.090687
Train Epoch: 0 [10/74]	loss: 4.057495
Train Epoch: 0 [20/74]	loss: 3.730631
Train Epoch: 0 [30/74]	loss: 3.363389
Train Epoch: 0 [40/74]	loss: 3.176519
Train Epoch: 0 [50/74]	loss: 2.856124
Train Epoch: 0 [60/74]	loss: 2.283674
Train Epoch: 0 [70/74]	loss: 2.457200
[0]Validation Loss: 1.9708,Accuracy: 63.8514%
Train Epoch: 1 [0/74]	loss: 1.925718
Train Epoch: 1 [10/74]	loss: 1.455559
Train Epoch: 1 [20/74]	loss: 1.510651
Train Epoch: 1 [30/74]	loss: 1.499170
Train Epoch: 1 [40/74]	loss: 1.172533
Train Epoch: 1 [50/74]	loss: 1.379685
Train Epoch: 1 [60/74]	loss: 1.093722
Train Epoch: 1 [70/74]	loss: 0.953686
[1]Validation Loss: 1.0702,Accuracy: 80.4054%
Train Epoch: 2 [0/74]	loss: 0.602690
Train Epoch: 2 [10/74]	loss: 0.587345
Train Epoch: 2 [20/74]	loss: 0.537681
Train Epoch: 2 [30/74]	loss: 0.384967
Train Epoch: 2 [40/74]	loss: 0.428245
Train Epoch: 2 [50/74]	loss: 0.517912
Train Epoch: 2 [60/74]	loss: 0.442037
Train Epoch: 2 [70/74]	loss: 0.366259